In [1]:

from utils import *

from LOTlib.SampleStream import *
from LOTlib.Inference.Samplers.MetropolisHastings import MHSampler
from LOTlib.DataAndObjects import FunctionData
from LOTlib.TopN import TopN
import copy

In [4]:




grammar = Grammar(start='SEQ')


grammar.add_rule('INT', "len", ["SEQ"], 1.0)
grammar.add_rule('INT', "mod", ["INT", "INT"], 1.0)
grammar.add_rule('INT', "minus", ["INT", "INT"], 1.0)
grammar.add_rule('INT', "plus", ["INT", "INT"], 1.0)



for i in xrange(0,9):
    grammar.add_rule('INT', str(i), None, 1.0)
grammar.add_rule('INT', str(INF), None, 5.0)

grammar.add_rule('SEQ', "'E'", None, 3.)

grammar.add_rule('SEQ', "'D'", None, 3.)

grammar.add_rule('SEQ', "'C'", None, 3.)
grammar.add_rule('SEQ', "'B'", None, 3.)
grammar.add_rule('SEQ', "'A'", None, 3.)


#grammar.add_rule('SEQ', "'9'", None, 0.1)
#grammar.add_rule('SEQ', "'8'", None, 0.25)
#grammar.add_rule('SEQ', "'7'", None, 0.1)
#grammar.add_rule('SEQ', "'6'", None, 0.25)
#grammar.add_rule('SEQ', "'5'", None, 0.1)
#grammar.add_rule('SEQ', "'4'", None, 0.25)
#grammar.add_rule('SEQ', "'3'", None, 0.1)
#grammar.add_rule('SEQ', "'2'", None, 0.25)
#grammar.add_rule('SEQ', "'1'", None, 0.1)

#grammar.add_rule('SEQ', 'flip(%s, %s, %s, %s)', ['INT'], 1.0) 
#grammar.add_rule('SEQ', 'all_rand(%s)', ['INT'], 1.0) 
#grammar.add_rule('SEQ', 'invert', ['SEQ'], 1.0) 
#grammar.add_rule('SEQ', "n_times(%s, %s)", ["SEQ", "INT"], 1.0)

grammar.add_rule('BOOL', "eq", ["SEQ", "SEQ"], 1.0)
grammar.add_rule('SEQ', "if_else", ["BOOL", "SEQ", "SEQ"], 1.0)
grammar.add_rule('SEQ', "alternate(%s, %s)", ["SEQ", "SEQ"], 1.0)
grammar.add_rule('SEQ', "repeat(%s)", ["SEQ"], 1.0)
grammar.add_rule('SEQ', "append(%s, %s)", ["SEQ", "SEQ"], 1.0)
grammar.add_rule('SEQ', 'upto', ['SEQ', 'INT'], 1.0) 
grammar.add_rule('SEQ', 'after', ['SEQ', 'INT'], 1.0) 
grammar.add_rule('SEQ', "cut(%s, %s, %s)", ["SEQ", "INT", "INT"], 1.0)
grammar.add_rule('SEQ', 'increment', ['SEQ', 'SEQ'], 0.5) 
grammar.add_rule('SEQ', 'mirror', ['SEQ'], 1.) 
grammar.add_rule('SEQ', 'expand', ['SEQ'], 1.) 
grammar.add_rule('SEQ', 'next_dig', ['SEQ'], 1.) 
grammar.add_rule('SEQ', 'prev_dig', ['SEQ'], 1.) 
grammar.add_rule('SEQ', 'index', ['SEQ', 'INT'], 1.) 

#grammar.add_rule('SEQ', '((%s)(%s))',  ['FUNCDEF', 'FUNC'], 1.0)
#grammar.add_rule('FUNCDEF', 'lambda', ['SEQ'], 1.0, bv_type='SEQ', bv_args=['SEQ'], bv_prefix='F')
#grammar.add_rule('FUNC', 'lambda', ['SEQ'], 1.0, bv_type='SEQ')



class MyHypothesis(LOTHypothesis):
    def __init__(self, **kwargs):

        self.start_counts={}
        LOTHypothesis.__init__(self, grammar=grammar,
        maxnodes=400, display='lambda from_seq: %s', **kwargs)
        #LOTHypothesis.__init__(self, grammar=grammar,
        #maxnodes=400, display='lambda x: %s', **kwargs)


    def __call__(self, *args):
        out = LOTHypothesis.__call__(self, *args)
        return out
    

    def compute_single_likelihood(self, datum):
        alpha = datum.alpha
        true_val = datum.output.keys()[0]
        generated = self(*datum.input)

        generated = generated[:len(true_val)]
        #dist = (hamming_distance(generated, true_val[:len(generated)]) + 
                  #   len(true_val) - len(generated))

        p = max(0,1+len(true_val) - len(generated)) * log(alpha)

        for i in xrange(0, min(len(generated), len(true_val))):
            v_g = generated[i]
            v_t = true_val[i]
            #if v_g not in "AB":
              #  if v_t == "B":
                    #p += log(float(v_g)/10.)
               # else:
                   # p += log(1 - float(v_g)/10.)
            #else:
            if v_g == v_t:
                p += log(1.-alpha)
            else:
                p += log(alpha)


        return p




In [14]:



ALPHA = 1e-8
STEPS = 5000
N_H = 25


for s in stim:

    data = [FunctionData(alpha=ALPHA,
         input=[""], output={"ABABBABBB":0})]
    

    h0 = MyHypothesis()
    seen = set()
    tn = TopN(N=N_H)


    for h in MHSampler(h0, data, 
            steps=STEPS, acceptance_temperature=1.): 
        out = h(s) 
        if out != "":
            print out


C
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
DCBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
DCBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBB

BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBB

ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBB

ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBB

ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBBBBBBBBBBABBBBBBBBBBBBB
ABBBBBBB

...

ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABA

ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABA

ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABA

ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABABBABABABBABBABABBABABA
ABABBABA

KeyboardInterrupt: 

In [5]:


ALPHA = 1e-8
STEPS = 5000
N_H = 25


stim = ["ABABACACA", "ABBACCCAD", "ABC", "AAABBB", "AAAABBBCCBBBAAAA", "ABBCCC", "ABABBABBB", "ABACAD"]
rule_key = get_rule_key(grammar)

lst = []
lst_all = []
count = 0
for s in stim:

    data = [FunctionData(alpha=ALPHA,
         input=[""], output={s: len(s)})]

    h0 = MyHypothesis()
    seen = set()
    tn = TopN(N=N_H)


    for h in MHSampler(h0, data, 
            steps=STEPS, acceptance_temperature=3.): 
        out = h(s) 
        if len(out) > len(s) and str(h) not in seen:
            seen.add(str(h))
            tn.add(h)

    h_count = 0
    top_hyps = tn.get_all(sorted=True)


    if len(top_hyps) != 25:
        print s
        print "AHHH NOT 25 OMG!"
        print "AHHH it's only ", len(top_hyps)
    for h in top_hyps:
        out= h(s)   
        print s, out[:20]
        print
        
        """
        l_rules = [str(i) for i in list(np.hstack(get_rule_counts(grammar, h.value)))]

        o_g = ""
        if len(out) > len(s):
            o_g = out[len(s)]
            print s
            print out
            print

            for i in xrange(len(rule_key)):
                gr = rule_key[i]
                gr_type = gr[0]
                gr_name = gr[1]
                gr_typename = "%s_%s" % gr
                lst_all.append((str(s),str(h_count), out[:len(s)], o_g,gr_type,gr_name,gr_typename,  
                                    l_rules[i],
                                str(h.posterior_score),
                                 str(h.prior), str(h.likelihood), out))
        """
        

        h_count += 1
    print count, s
    print "*" * 75
    print "*" * 75

    count += 1




output1(lst, "output/out1.csv")

output2(lst_all, "output/out2.csv")




ABABACACA ACACACACACACACACACAC

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

ABABACACA ACACACACACACACACACAC

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ABABABABABABABABABAB

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

ABABACACA ABABABABABABABABABAB

ABABACACA ACACACACACACACACACAC

0 ABABACACA
***************************************************************************
***************************************************************************
ABBACCCAD ADBACCCADBACCCADBACC

ABBA

KeyboardInterrupt: 